In [2]:
import os
os.chdir('..')
%pwd

'd:\\Programming\\MachineLearning\\MLOPs\\End to End ML PipeLine'

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [4]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    test_data_path: Path
    all_param: dict
    metric_file_path: Path
    target_column: str

In [5]:
from MLOps.constants import *
from MLOps.utils.common import read_yaml, create_directories,save_json

In [6]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH, schema_file_path=SCHEMA_FILE_PATH):
        print(f"ConfigurationManager initialized with config: {config_file_path}, params: {params_file_path}, schema: {schema_file_path}")
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories(list_of_directories=[self.config.artifacts_root])
        
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            test_data_path=config.test_data_path,
            all_param=params,
            metric_file_path=config.metric_file_path,
            target_column=self.schema.TARGET_COLUMN.name
        )
        print(model_evaluation_config)
        
        return model_evaluation_config

In [7]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import numpy as np,pandas as pd
import joblib

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        self.model = joblib.load(self.config.model_path)
        self.test_data = pd.read_csv(self.config.test_data_path)
        self.target_column = self.config.target_column
        
    def evaluate_model(self,y_act,y_pred):
        rmse = np.sqrt(mean_squared_error(y_act,y_pred))
        mae = mean_absolute_error(y_act,y_pred)
        r2 = r2_score(y_act,y_pred)
        return rmse,mae,r2

    def evaluate(self):
        X_test = self.test_data.drop(columns=[self.target_column])
        y_test = self.test_data[self.target_column]
        
        y_pred = self.model.predict(X_test)
        
        rmse, mae, r2 = self.evaluate_model(y_test, y_pred)
        
        metrics = {
            "rmse": rmse,
            "mae": mae,
            "r2": r2
        }
        
        save_json(self.config.metric_file_path,metrics,)

In [8]:
from MLOps import logger
try:
    logger.info(">>>>>>>>> stage Model Evaluation Stage started <<<<<<<<")
    config_manager = ConfigurationManager()
    model_evaluation_config = config_manager.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
    logger.info(">>>>>>>>> stage Model Evaluation Stage completed <<<<<<<<")
except Exception as e:
    logger.error(f"Error in Model Evaluation Stage: {e}")
    raise e

2025-07-02 04:27:26,313 - MLOpsLogger - INFO - >>>>>>>>> stage Model Evaluation Stage started <<<<<<<<
ConfigurationManager initialized with config: config\config.yaml, params: params.yaml, schema: schema.yaml
Directory already exists: artifacts
ModelEvaluationConfig(root_dir='artifacts/model_evaluation', model_path='artifacts/model_trainer/model.joblib', test_data_path='artifacts/data_transformation/test.csv', all_param=ConfigBox({'alpha': 0.1, 'l1_ratio': 0.01, 'random_state': 12}), metric_file_path='artifacts/model_evaluation/metrics.json', target_column='quality')
JSON data saved to: artifacts\model_evaluation\metrics.json
2025-07-02 04:27:26,367 - MLOpsLogger - INFO - >>>>>>>>> stage Model Evaluation Stage completed <<<<<<<<
